# Data cleaning
##  1. Concatenate Users
We have two important samples. One from January of 2021 and other for October of 2019. This script imports the rts made by each user in our sample, one by one, and later combine all the retweets in a pickle.

In [22]:
import os
import pandas as pd
import numpy as np
import glob
from tqdm import tqdm

In [2]:
# We create an empty aux list that will store the tweets.
tweets_aux = []

### Import January 21' tweets
[comment]: <> (Duda #1. En el archivo original el path es "../../Data/Checks/28-06-23" pero ahí solo hay .pkl)

We identify two users with their file corrupted: Usuario_82383620 and Usuario_2526574133

In [3]:
files_jan = glob.glob('../../../Data/users_jan/*.csv')

for file in tqdm(files_jan):
    tweets_aux.append(pd.read_csv(file))

100%|██████████| 34048/34048 [03:16<00:00, 173.16it/s]


### Import October 19' tweets

We identify one user with their file corrupted: Usuario_1153335229597278208

In [4]:
files_oct = glob.glob('../../../Data/users_oct_19/*.csv')

for file in tqdm(files_oct):
    tweets_aux.append(pd.read_csv(file))

100%|██████████| 25125/25125 [02:49<00:00, 147.99it/s]


In [5]:
# Finally, the tweet dataframe is established and tweets_aux is deleted.  
tweets = pd.concat(tweets_aux)
del tweets_aux
tweets = tweets.sort_values('ID').reset_index(drop = True)

In [19]:
tweets.shape

(11317934, 25)

In [11]:
# Store results
tweets.to_pickle("../Data/tweets_oct19_jan21.gzip", compression = "gzip")

---
## 2. Assign political affiliation based on retweets
This section determines the political affiliation (left, center, right) of each user in our sample by analyzing the retweets they've made.

We use a list of political influencers that have been previously categorized as left, center, or right by La Silla Vacia, a Colombian news outlet. For each user, we tally the number of retweets (excluding those with comments) corresponding to each influencer. From this, we calculate the total number of tweets associated with each political category.

In [15]:
# Let's load the id_to_party_map pickle file. This pickle contains some labels for political influencers in Colombia. 
# These were assigned using "La Silla Vacia" research.
mapa = pd.read_pickle("../Data/mapa.pkl")

In [23]:
# Now we assign each RT a political label according to its influencer's label.
tweets["Party"] = np.nan
tweets.loc[tweets["Reference Type"] == "retweeted", "Party"] = tweets.loc[tweets["Reference Type"] == "retweeted",
                                                                         "Referenced Tweet Author ID"].map(mapa)

In [26]:
# Store results
tweets.to_pickle("../Data/tweets_oct19_jan21.gzip", compression = "gzip")